# Project 2 Data sources:

## Observed dependent variable:
* Drug-induced mortality per county for 2016
* reach: opioid-induced mortality per county for 2016
* double-reach: opioid-induced mortality per county over 5 years, 2011 - 2016

## Independent variables/regressors:

### Medical:
* Age of electronic PDMP per state
* Number of methadone clinics per county
* Prescribing map -> rate of opioid prescriptions per 100 US residents

### Legislative:
* Number of bills per state that contain the word 'opioid'

### Economic:
* Poverty
* Percent unemployment
* Median salary

### Demographic:
* Median age
* Percentage minority
* Education - percentage high school grad or higher

Websites for data sources:
* https://wonder.cdc.gov/
* https://www.bls.gov/lau/data.htm
* https://censusreporter.org/

### Get list of counties

First, get a list of all of the counties along with their states and INCITS from this page: https://en.wikipedia.org/wiki/List_of_United_States_counties_and_county_equivalents

In [103]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import pickle
import time

from bs4 import BeautifulSoup

In [104]:
url = "https://en.wikipedia.org/wiki/List_of_United_States_counties_and_county_equivalents"
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "html5lib")

In [105]:
table = soup.select_one('table[class*="wikitable"]')
rows = [row for row in table.find_all("tr")]

In [106]:
# Remove the first <tr> element, which is the header row for the wikitable.
rows = rows[1:]

county_df = pd.DataFrame(columns=['INCITS', 'county_name', 'state'])

for row in rows:
    elements = row.find_all("td")
    elements = [element.text.strip() for element in elements]
    county_df = county_df.append({'INCITS': elements[0], 'county_name': elements[1], 'state': elements[2]}, ignore_index=True)

In [107]:
county_df.head()

,INCITS,county_name,state
0,01001,Autauga County,Alabama
1,01003,Baldwin County,Alabama
2,01005,Barbour County,Alabama
3,01007,Bibb County,Alabama
4,01009,Blount County,Alabama


In [6]:
county_df['state'].unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaiʻi',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Northern Mariana Islands',
       'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'U.S. Minor Outlying Islands', 'Utah', 'Vermont',
       'Virgin Islands (U.S.)', 'Virginia', 'Washington', 'West Virginia',
       'Wisconsin', 'Wyoming'], dtype=object)

In [7]:
# There are 'states' included that are not actually states. Let's make a list of them.
not_states = ['American Samoa', 'Guam', 'Northern Mariana Islands', 'Puerto Rico', 'U.S. Minor Outlying Islands', 'Virgin Islands (U.S.)']

In [8]:
# Now, let's remove all of the entries for these 'not states' from the data frame.
county_df = county_df[~county_df.state.isin(not_states)]

In [47]:
# Confirm that there are only states and the District of Columbia left.
states = county_df['state'].unique()
states[11] = "Hawaii"

In [31]:
county_df[county_df.INCITS == "02270"]
# Per google, Kusilvak Census area changed its INCITS to 02158 in 2015.
# Will need to change this manually; wikipedia was out of date.

,INCITS,county_name,state
92,02270,Kusilvak Census Area[10],Alaska


In [45]:
row = county_df.loc[county_df.INCITS == "02270"]
county_df.at[row.index[0], 'INCITS']

'02270'

In [46]:
# Correct this field manually.
county_df.at[row.index[0], 'INCITS'] = "02158"
county_df.at[row.index[0], 'INCITS']

'02158'

In [48]:
# save county dataframe to pickle.
with open("county_df.pkl", "wb") as picklefile:
    pickle.dump([county_df, states], picklefile)

### Economic and Demograhic data

Let's scraping economic and demographic data from census reporter; this website provides data at a state and county level based on the ACS (American Communities Survey) 2016 5-year data, which is provided by the US Census.

The more in-depth way to do this would be to figure out how to use the US Census API to get the 6 variables I specified, because with the US Census API you can fully specify what dataset to query and what parameters to return, but as an intermediate step I'm going to scrape CensusReporter.com first since those pages easier to access and scrape.

In [12]:
# For the census reporter page, each county page is structured so that the stats are located in spans with either the class value or name.

def get_stats_from_soup(incit, soup):
    row_dict = {'INCITS': incit}
    stats = soup.select('a[class*=stat]')
    for stat in stats:
        val = stat.select_one('span[class="value"]').text
        whitespace = r'[\s\t\n]*'
        val = re.sub(whitespace, "", val) # There's a lot of weird whitespace in the class=value element, so strip all of it out.
    
        val = val.split("±")[0] # A few of these stats have +/- ranges on them; we only want the main value, so split and get the first element.
        regex = r'([a-zA-Z\$,\%])*'
        val = re.sub(regex, "", val) # Strip out all the letters, dollar signs, commas, and percentage signs as well.
    
        name = stat.select_one('span[class="name"]').text.strip()
    
        if name in census_reporter_dict:
            row_dict[census_reporter_dict[name]] = val
    
    return row_dict

In [18]:
census_reporter_dict = ({"Median age": "median_age", 
                         "Median household income": "median_hh_income", 
                         "Persons below poverty line": "poverty_percent",
                         "High school grad or higher": "hs_percent"
                        })

cols = ['INCITS']
cols.extend(list(census_reporter_dict.values()))
census_reporter_df = pd.DataFrame(columns = cols)

In [29]:
%%time
census_reporter_query = "https://censusreporter.org/profiles/05000US{}"

mini_county_df = county_df.iloc[0:100]

for incit in mini_county_df['INCITS']:
    response = requests.get(census_reporter_query.format(incit))
    page = response.text
    soup = BeautifulSoup(page, "html5lib")
    census_reporter_df = census_reporter_df.append(get_stats_from_soup(incit, soup), ignore_index=True)
    time.sleep(1)

CPU times: user 15.8 s, sys: 1.47 s, total: 17.3 s
Wall time: 4min 57s


In [30]:
census_reporter_df.iloc[90:]

,INCITS,median_age,median_hh_income,poverty_percent,hs_percent
90,01001,37.8,53099,12.3,87.6
91,01003,42.4,56732,11.4,89.9
92,01005,38.7,33956,26.4,73.8
93,01007,40.2,39776,16.5,80.7
94,01009,40.8,46212,16.5,80
95,01011,39.2,29335,26.9,66.6
96,01013,40.6,34315,25.7,81.1
97,01015,39.1,41687,16.3,82.2
98,01017,43.1,36027,20.3,80.3
99,01019,45.7,38925,16.5,81.3


In [11]:
# Let's pull the relevant parameters for Barbour County.
# For economic parameters, I want % poverty, % unemployment, and median salary.
# For demographics, I want median age, percentage minority, and education (percentage high school grad)
# We can go ahead and print all of the stats within any element that belongs to a class w/ the word 'stat'.
    
stats = soup.select('a[class*="stat"]')
for stat in stats:
    val = stat.select_one('span[class="value"]').text
    whitespace = r'[\s\t\n]*'
    val = re.sub(whitespace, "", val) # There's a lot of weird whitespace in the class=value element, so strip all of it out.
    
    name = stat.select_one('span[class="name"]').text.strip()
    
    print("{} is {}".format(name, val))
    
    val2 = val.split("±")[0]
    regex = r'([a-zA-Z\$,\%])*'
    print(re.sub(regex, "", val2))

Median age is 38.7±0.6
38.7
Per capita income is $17,249±$822
17249
Median household income is $33,956±$2,655
33956
Persons below poverty line is 26.4%±2.7%(6,235±636)
26.4
Mean travel time to work is 23.7minutes±1.7(205,890±17,933)
23.7
Number of households is 9,122±286
9122
Persons per household is 2.6±0.1(23,682±244)
2.6
Women 15-50 who gave birth during past year is 5%±1.8%(252±92)
5
Number of housing units is 11,802±101
11802
Median value of owner-occupied housing units is $90,300±$7,258
90300
Moved since previous year is 12.4%±1.4%(3,262±373.4)
12.4
High school grad or higher is 73.8%±3.1%(13,563±573.2)
73.8
Bachelor's degree or higher is 12.9%±1.5%(2,366±270.8)
12.9
Persons with language other than English spoken at home is N/A
/
Foreign-born population is 2.9%±0.3%(761±82)
2.9
Population with veteran status is 8.4%±0.8%(1,751±173)
8.4
Total veterans is 1,751
1751


In [32]:
def get_stats_from_soup(incit, soup):
    row_dict = {'INCITS': incit}
    stats = soup.select('a[class*=stat]')
    for stat in stats:
        val = stat.select_one('span[class="value"]').text
        whitespace = r'[\s\t\n]*'
        val = re.sub(whitespace, "", val) # There's a lot of weird whitespace in the class=value element, so strip all of it out.
    
        val = val.split("±")[0] # A few of these stats have +/- ranges on them; we only want the main value, so split and get the first element.
        regex = r'([a-zA-Z\$,\%])*'
        val = re.sub(regex, "", val) # Strip out all the letters, dollar signs, commas, and percentage signs as well.
    
        name = stat.select_one('span[class="name"]').text.strip()
    
        if name in census_reporter_dict:
            row_dict[census_reporter_dict[name]] = val
    
    return row_dict

In [48]:
census_reporter_query = "https://censusreporter.org/profiles/05000US{}"

census_reporter_dict = ({"Median age": "median_age", 
                         "Median household income": "median_hh_income", 
                         "Persons below poverty line": "poverty_percent",
                         "High school grad or higher": "hs_percent"
                        })

cols = ['INCITS']
cols.extend(list(census_reporter_dict.values()))
census_reporter_df = pd.DataFrame(columns = cols)

mini_county_df = county_df.iloc[0:10]

for incit in mini_county_df['INCITS']:
    response = requests.get(census_reporter_query.format(incit))
    page = response.text
    soup = BeautifulSoup(page, "html5lib")
    census_reporter_df = census_reporter_df.append(get_stats_from_soup(incit, soup), ignore_index=True)

In [51]:
census_reporter_df

,INCITS,median_age,median_hh_income,poverty_percent,hs_percent
0,01001,37.8,53099,12.3,87.6
1,01003,42.4,56732,11.4,89.9
2,01005,38.7,33956,26.4,73.8
3,01007,40.2,39776,16.5,80.7
4,01009,40.8,46212,16.5,80
5,01011,39.2,29335,26.9,66.6
6,01013,40.6,34315,25.7,81.1
7,01015,39.1,41687,16.3,82.2
8,01017,43.1,36027,20.3,80.3
9,01019,45.7,38925,16.5,81.3


In [61]:
mini_county_df = county_df.iloc[0:10]

In [62]:
print(mini_county_df)
print(census_reporter_df)

  INCITS      county_name    state
0  01001   Autauga County  Alabama
1  01003   Baldwin County  Alabama
2  01005   Barbour County  Alabama
3  01007      Bibb County  Alabama
4  01009    Blount County  Alabama
5  01011   Bullock County  Alabama
6  01013    Butler County  Alabama
7  01015   Calhoun County  Alabama
8  01017  Chambers County  Alabama
9  01019  Cherokee County  Alabama
  INCITS median_age median_hh_income poverty_percent hs_percent
0  01001       37.8            53099            12.3       87.6
1  01003       42.4            56732            11.4       89.9
2  01005       38.7            33956            26.4       73.8
3  01007       40.2            39776            16.5       80.7
4  01009       40.8            46212            16.5         80
5  01011       39.2            29335            26.9       66.6
6  01013       40.6            34315            25.7       81.1
7  01015       39.1            41687            16.3       82.2
8  01017       43.1            36027   

In [67]:
combo = pd.merge(mini_county_df, census_reporter_df, on="INCITS")


In [68]:
combo

,INCITS,county_name,state,median_age,median_hh_income,poverty_percent,hs_percent
0,01001,Autauga County,Alabama,37.8,53099,12.3,87.6
1,01003,Baldwin County,Alabama,42.4,56732,11.4,89.9
2,01005,Barbour County,Alabama,38.7,33956,26.4,73.8
3,01007,Bibb County,Alabama,40.2,39776,16.5,80.7
4,01009,Blount County,Alabama,40.8,46212,16.5,80
5,01011,Bullock County,Alabama,39.2,29335,26.9,66.6
6,01013,Butler County,Alabama,40.6,34315,25.7,81.1
7,01015,Calhoun County,Alabama,39.1,41687,16.3,82.2
8,01017,Chambers County,Alabama,43.1,36027,20.3,80.3
9,01019,Cherokee County,Alabama,45.7,38925,16.5,81.3


## County FIPS codes from census

This is loading the FIPS codes from the census; it's probably more accurate than wikipedia, hopefully.

In [147]:
file_name = "all-geocodes-v2016.xlsx"

import pandas as pd

df = pd.read_excel(io=file_name)

In [148]:
df.head()

,Estimates Geography File: Vintage 2016,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN
1,Internet Release Date: May 2017,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area de...
4,010,00,000,00000,00000,00000,United States


In [158]:
desired_column_names = list(df.iloc[3])
current_column_names = df.columns
col_dict = dict(zip(current_column_names, desired_column_names))
col_dict

{'Estimates Geography File: Vintage 2016': 'Summary Level',
 'Unnamed: 1': 'State Code (FIPS)',
 'Unnamed: 2': 'County Code (FIPS)',
 'Unnamed: 3': 'County Subdivision Code (FIPS)',
 'Unnamed: 4': 'Place Code (FIPS)',
 'Unnamed: 5': 'Consolidtated City Code (FIPS)',
 'Unnamed: 6': 'Area Name (including legal/statistical area description)'}

In [159]:
clean_df = df.iloc[4:]
clean_df = clean_df.rename(col_dict, axis='columns')

In [160]:
clean_df.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
4,010,00,000,00000,00000,00000,United States
5,040,01,000,00000,00000,00000,Alabama
6,050,01,001,00000,00000,00000,Autauga County
7,050,01,003,00000,00000,00000,Baldwin County
8,050,01,005,00000,00000,00000,Barbour County


In [161]:
county_df = clean_df[clean_df['Summary Level'] == '050']
state_df = clean_df[clean_df['Summary Level'] == '040']

In [162]:
county_df.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
6,050,01,001,00000,00000,00000,Autauga County
7,050,01,003,00000,00000,00000,Baldwin County
8,050,01,005,00000,00000,00000,Barbour County
9,050,01,007,00000,00000,00000,Bibb County
10,050,01,009,00000,00000,00000,Blount County


In [165]:
county_df.tail()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
43933,050,72,145,00000,00000,00000,Vega Baja Municipio
43934,050,72,147,00000,00000,00000,Vieques Municipio
43935,050,72,149,00000,00000,00000,Villalba Municipio
43936,050,72,151,00000,00000,00000,Yabucoa Municipio
43937,050,72,153,00000,00000,00000,Yauco Municipio


In [164]:
state_df

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
5,040,01,000,00000,00000,00000,Alabama
534,040,02,000,00000,00000,00000,Alaska
712,040,04,000,00000,00000,00000,Arizona
819,040,05,000,00000,00000,00000,Arkansas
1397,040,06,000,00000,00000,00000,California
1938,040,08,000,00000,00000,00000,Colorado
2274,040,09,000,00000,00000,00000,Connecticut
2483,040,10,000,00000,00000,00000,Delaware
2544,040,11,000,00000,00000,00000,District of Columbia
2547,040,12,000,00000,00000,00000,Florida


In [169]:
# We are going to drop Puerto Rico from both the state and county dataframes.
county_df = county_df[county_df["State Code (FIPS)"] != "72"]
state_df = state_df[state_df["State Code (FIPS)"] != "72"]

In [170]:
county_df.shape

(3142, 7)